In [1]:
import numpy as np
import pandas as pd
import yt_dlp
from tqdm import tqdm
import os
import glob
import json

In [2]:
MEDIA_PATH = '/mnt/d/AI Challenge/Data/media-info'
DOWNLOAD_FOLDER = '/mnt/d/AI Challenge/Data/video'
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)
part_index = 0 # Dat, Huan, Tuan, Phat, Khoa

In [3]:
video_list = glob.glob(f"{MEDIA_PATH}/*.json")

# List to hold data
data_list = []
ignore_fields = ["description", "keywords", "title"]

for file in video_list:
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)  # Load JSON content
        data['name'] = os.path.basename(file).replace('.json', '')
        for ignore_field in ignore_fields:
            if ignore_field in data:
                del data[ignore_field]
        
        data_list.append(data)

df = pd.DataFrame(data_list)
df.set_index('name', inplace=True)
df.head()

,author,channel_id,channel_url,length,publish_date,thumbnail_url,watch_url
name,,,,,,,
L21_V001,60 Giây Official,UCRjzfa1E0gA50lvDQipbDMg,https://www.youtube.com/channel/UCRjzfa1E0gA50...,1262,01/08/2024,https://i.ytimg.com/vi/Rzpw5WR7nAY/sddefault.j...,https://youtube.com/watch?v=Rzpw5WR7nAY
L21_V002,60 Giây Official,UCRjzfa1E0gA50lvDQipbDMg,https://www.youtube.com/channel/UCRjzfa1E0gA50...,1057,02/08/2024,https://i.ytimg.com/vi/qqC4FLMNLdI/sddefault.j...,https://youtube.com/watch?v=qqC4FLMNLdI
L21_V003,60 Giây Official,UCRjzfa1E0gA50lvDQipbDMg,https://www.youtube.com/channel/UCRjzfa1E0gA50...,1198,03/08/2024,https://i.ytimg.com/vi/JrDeTiLY0u8/sddefault.j...,https://youtube.com/watch?v=JrDeTiLY0u8
L21_V005,60 Giây Official,UCRjzfa1E0gA50lvDQipbDMg,https://www.youtube.com/channel/UCRjzfa1E0gA50...,943,05/08/2024,https://i.ytimg.com/vi/GT5KeMmD5Sw/sddefault.j...,https://youtube.com/watch?v=GT5KeMmD5Sw
L21_V006,60 Giây Official,UCRjzfa1E0gA50lvDQipbDMg,https://www.youtube.com/channel/UCRjzfa1E0gA50...,1036,06/08/2024,https://i.ytimg.com/vi/HphV3ZNfA5A/sddefault.j...,https://youtube.com/watch?v=HphV3ZNfA5A


In [4]:
df_parts = np.array_split(df, 5)

# # Access each part
# for i, part in enumerate(df_parts, 0):
#     print(f"Part {i + 1}:\n", part.index[0], part.index[-1], "\n")

df = df_parts[part_index] # select part here

/home/datlinux/Video-Event-Retrieval-AI-Challenge-2025/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
MAX_WORKERS = 4

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed


# Common download options
base_opts = {
    'format': 'mp4',
    'quiet': True,
    'writesubtitles': True,
    'subtitleslangs': ['vi'],
    'writeautomaticsub': True,
}

def download_video(name, row):
    batchh = name.split("_")[0]  # sanitize filename
    output_path = f"{DOWNLOAD_FOLDER}/{batchh}/{name}.%(ext)s"

    if os.path.exists(output_path):
        return f"Video {name} already exists, skipping."

    ydl_opts = {**base_opts, 'outtmpl': output_path}

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([row['watch_url']])
        return f"Downloaded {name}"
    except Exception as e:
        return f"Failed {name}: {e}"

# Number of parallel threads
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(download_video, name, row) for name, row in df.iterrows()]
    for f in tqdm(as_completed(futures), total=len(futures), desc="Downloading videos"):
        print(f.result())

[download]    3.00KiB at   64.65KiB/s (00:00:00)

[download]  25.0% of   79.00MiB at    1.23MiB/s ETA 00:48  

[download]  27.3% of   79.00MiB at    7.26MiB/s ETA 00:07

[download]  88.4% of   79.00MiB at    1.29MiB/s ETA 00:07  

[download]  22.9% of   76.47MiB at   31.06MiB/s ETA 00:01  

[download]  15.7% of   62.30MiB at  790.40KiB/s ETA 01:08  

[download]  76.3% of   76.47MiB at   11.73MiB/s ETA 00:01